In [1]:
import pandas as pd
import sqlite3

In [8]:
def get_db_connection():
    """Create a connection to the SQLite database"""
    db_path = "/mnt/disk1/Github/Dash_Imagination/src/dash_imagination/data/imagination.db"
    
    print(f"Connecting to database at: {db_path}")
    
    # Connect to the database
    conn = sqlite3.connect(db_path)
    conn.row_factory = sqlite3.Row  # Access columns by name
    return conn

def pdquery(conn, query, params=()):
    """Execute a query and return results as DataFrame"""
    return pd.read_sql_query(query, conn, params=params)

def get_authors():
    """Get list of authors from the database"""
    conn = get_db_connection()
    df = pdquery(conn, "SELECT DISTINCT author FROM corpus WHERE author IS NOT NULL ORDER BY author")
    conn.close()
    # Convert to plain Python list and filter out None values
    authors = [str(author) for author in df['author'].tolist() if author is not None]
    return authors

def get_categories():
    """Get list of categories from the database"""
    conn = get_db_connection()
    df = pdquery(conn, "SELECT DISTINCT category FROM corpus WHERE category IS NOT NULL ORDER BY category")
    conn.close()
    # Convert to plain Python list and filter out None values
    categories = [str(category) for category in df['category'].tolist() if category is not None]
    return categories

def get_titles():
    """Get list of book titles from the database"""
    conn = get_db_connection()
    df = pdquery(conn, "SELECT DISTINCT title, year FROM corpus WHERE title IS NOT NULL ORDER BY title")
    
    # Create title_year safely
    title_year_list = []
    for _, row in df.iterrows():
        title = row['title']
        year = row['year']
        if title is not None:
            year_str = f"({year})" if year is not None else "(n.d.)"
            title_year_list.append(f"{title} {year_str}")
    
    conn.close()
    return title_year_list

def get_places_for_map(filters=None):
    """Get places data filtered by user selections"""
    conn = get_db_connection()
    
    base_query = """
    SELECT 
        p.token, 
        p.modern as name,
        p.latitude, 
        p.longitude, 
        SUM(bp.frequency) as frequency,
        COUNT(DISTINCT bp.dhlabid) as book_count
    FROM 
        places p
    JOIN 
        book_places bp ON p.token = bp.token
    JOIN 
        corpus c ON bp.dhlabid = c.dhlabid
    """
    
    where_clauses = []
    params = []
    
    if filters:
        if 'year_range' in filters and filters['year_range']:
            where_clauses.append("c.year BETWEEN ? AND ?")
            params.extend([filters['year_range'][0], filters['year_range'][1]])
        
        if 'categories' in filters and filters['categories']:
            placeholders = ','.join(['?'] * len(filters['categories']))
            where_clauses.append(f"c.category IN ({placeholders})")
            params.extend(filters['categories'])
        
        if 'authors' in filters and filters['authors']:
            placeholders = ','.join(['?'] * len(filters['authors']))
            where_clauses.append(f"c.author IN ({placeholders})")
            params.extend(filters['authors'])
            
        if 'titles' in filters and filters['titles']:
            # Extract just the title part without the year
            title_only = [t.split(' (')[0] for t in filters['titles']]
            placeholders = ','.join(['?'] * len(title_only))
            where_clauses.append(f"c.title IN ({placeholders})")
            params.extend(title_only)
    
    # Add WHERE clause if we have any filters
    if where_clauses:
        base_query += " WHERE " + " AND ".join(where_clauses)
    
    # Add grouping and limit
    base_query += """
    GROUP BY p.token, p.modern, p.latitude, p.longitude
    ORDER BY frequency DESC
    LIMIT ?
    """
    
    # Add the limit parameter (default to 200 if not specified)
    max_places = filters.get('max_places', 200) if filters else 200
    params.append(max_places)
    
    try:
        df = pdquery(conn, base_query, tuple(params))
        conn.close()
        return df
    except Exception as e:
        print(f"Error querying database: {e}")
        conn.close()
        # Return empty DataFrame with correct columns
        return pd.DataFrame(columns=['token', 'name', 'latitude', 'longitude', 'frequency', 'book_count'])

def get_place_details(token):
    """Get details for a specific place including books where it appears"""
    conn = get_db_connection()
    
    # Query to get books containing this place
    query = """
    SELECT DISTINCT c.title, c.author, c.year, c.urn, bp.token, bp.frequency
    FROM corpus c
    JOIN book_places bp ON c.dhlabid = bp.dhlabid
    WHERE bp.token = ?
    ORDER BY bp.frequency DESC
    LIMIT 20
    """
    
    books = pdquery(conn, query, (token,))
    conn.close()
    
    return books

In [8]:
pdquery(get_db_connection(), "select * from sqlite_master").style

Connecting to database at: /mnt/disk1/Github/Dash_Imagination/src/dash_imagination/data/imagination.db


,type,name,tbl_name,rootpage,sql
0,table,corpus,corpus,4464,"CREATE TABLE ""corpus"" ( ""author"" TEXT, ""category"" TEXT, ""oversatt"" INTEGER, ""place"" TEXT, ""publisher"" TEXT, ""title"" TEXT, ""urn"" TEXT, ""year"" INTEGER, ""dhlabid"" INTEGER )"
1,table,places,places,2,"CREATE TABLE ""places"" ( ""token"" TEXT, ""modern"" TEXT, ""type"" TEXT, ""area"" TEXT, ""latitude"" REAL, ""longitude"" REAL, ""frequency"" INTEGER )"
2,index,_t_,places,14192,CREATE INDEX _t_ on places(token)
3,index,_auth_,corpus,19390,"CREATE INDEX _auth_ on corpus(author, dhlabid)"
4,index,_dhlabid_,corpus,25851,CREATE INDEX _dhlabid_ on corpus(dhlabid)
5,table,book_places,book_places,1003,"CREATE TABLE ""book_places"" ( ""token"" TEXT, ""frequency"" INTEGER, ""latitude"" REAL, ""longitude"" REAL, ""dhlabid"" INTEGER )"
6,index,_a_,book_places,22612,CREATE INDEX _a_ on book_places(token)
7,index,_place_,book_places,29794,"CREATE INDEX _place_ on book_places(dhlabid, token)"


In [3]:
get_categories()

Connecting to database at: /mnt/disk1/Github/Dash_Imagination/src/dash_imagination/data/imagination.db


['Barnelitteratur',
 'Biografi / memoar',
 'Diktning: Dramatikk',
 'Diktning: Dramatikk # Diktning: oversatt',
 'Diktning: Epikk',
 'Diktning: Epikk # Diktning: oversatt',
 'Diktning: Lyrikk',
 'Diktning: Lyrikk # Diktning: oversatt',
 'Diverse',
 'Filosofi / estetikk / språk',
 'Historie / geografi',
 'Lesebok / skolebøker / pedagogikk',
 'Litteraturhistorie / litteraturkritikk',
 'Naturvitenskap / medisin',
 'Reiselitteratur',
 'Religiøse / oppbyggelige tekster',
 'Samfunn / politikk / juss',
 'Skisser / epistler / brev / essay / kåseri',
 'Taler / sanger / leilighetstekster',
 'Teknologi / håndverk / landbruk / havbruk']

In [11]:
%%time
get_place_details("Usa")

Connecting to database at: /mnt/disk1/Github/Dash_Imagination/src/dash_imagination/data/imagination.db
CPU times: user 5.63 ms, sys: 1.37 ms, total: 7 ms
Wall time: 5.61 ms


,title,author,year,urn,token,frequency
0,Bibelen eller Den hellige Skrift : paany overs...,"Kalkar, Chr. H.",1847,URN:NBN:no-nb_digibok_2015061048039,Usa,4
1,Kongesagaer,"Snorri Sturluson / Storm, Gustav / Egedius, Ha...",1899,URN:NBN:no-nb_digibok_2016050948137,Usa,2
2,Kongesagaer,"Snorri Sturluson / Egedius, Halfdan / Krohg, C...",1899,URN:NBN:no-nb_digibok_2009052803028,Usa,2
3,Heimskringla eller Norges Konge-Sagaer,"Snorri Sturluson / Munch, P.A. (Peter Andreas)...",1897,URN:NBN:no-nb_digibok_2009101310001,Usa,2
4,Dansk-norsk-engelsk Ordbog,"Larsen, A.",1897,URN:NBN:no-nb_digibok_2013081208214,Usa,2
5,Norges Konge-Sagaer : fra de ældste Tider indt...,None,1859,URN:NBN:no-nb_digibok_2006112101078,Usa,1
6,Norges kongesagaer. 1-2 2 : Heimskringla,"Snorri Sturluson / Storm, Gustav / Munthe, Ger...",1899,URN:NBN:no-nb_digibok_2012091108000,Usa,1
7,Norges kongesagaer. 1-2 1 : Heimskringla,"Snorri Sturluson / Storm, Gustav / Munthe, Ger...",1899,URN:NBN:no-nb_digibok_2012090408089,Usa,1
8,Tydsk-dansk og dansk-tydsk Haand-Ordbog. D. 2 ...,"Grønberg, B.C.",1851,URN:NBN:no-nb_digibok_2008091603033,Usa,1
9,Jómsvikingasaga ok Knytlingasaga með tilheyran...,None,1828,URN:NBN:no-nb_digibok_2016060948020,Usa,1
